### Using k-d tree for distance problems

![KDTREE Conceptual.png](Assets/KDTREE_Conceptual.png  "Conceptual view of KDTree from Wikipedia")

KDTrees, also known as k-d trees, consist of a mechanisms of inserting spatial data into a k dimensions tree at insertion time, and allowing distance calculations based on locations within the tree at a later read time. The average time complexity for a nearest neighbor type search is ~log(n) which is a considerable improvement over brute force methods. Essentially, this method of storing the points away in a tree structure means that when looking for nearby points you already know what part of the tree to search.

For more detail on k-d tree see wiki [k-d tree](https://en.wikipedia.org/wiki/K-d_tree) 

# Use k-d tree to find all points within radius R

k-d tree is not directly optimized by Intel at this point but is used under the hood by Intel's kNN which is optimized.

Since we are dealing with a fairly large number of points (more than a few thousand) let's explore the inherent data structure used by the kNN under the hood - k-d tree to see how quickly we can find pairs of points within radius R of each other.

This mechanism is very efficient and k-d tree algorithms have this estimate of computation and space complexity.
[wikipedia k-d tree](https://en.wikipedia.org/wiki/K-d_tree):

![Complexity](Assets/KDTREE_Complexity.png)

Below we find the all nearly colliding stars (here ww will say stars within 3 light years from each other). This could answer the question: **Did these galaxies result in more "tooClose" encounters after the collision?**

#### k-d tree
On a CPU, in Intel Extension for Scikit-learn, the library provides kNN classification based on multidimensional binary search tree (k-d tree, where d means the dimension and k means the number of dimensions in the feature space). 

For more details on Intel's implementation of kNN using k-d tree, see [k-Nearest Neighbors (kNN) Classifier](https://oneapi-src.github.io/oneDAL/daal/algorithms/k_nearest_neighbors/k-nearest-neighbors-knn-classifier.html) which has more discussion of kNN and the underlying kdtree approach.

In this video you can see both the result of the kNN classification (each galaxy is different color) as well as observing the stars detected within a radius R as a result of the collision plotted in red after the other stars as gradually changed to transparent see the near-collision at the center.
 

<video controls src="./Videos/CollidingGalaxiesAnimation.mp4" />


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import time
from random import sample
from sklearn.neighbors import KDTree

def read_dictionary(fn):
    import pickle
    # Load data (deserialize)
    with open(fn, 'rb') as handle:
        dictionary = pickle.load(handle)
    return dictionary

def add(arr, myset):
    for pt3D in arr:
        mylist = []
        for pt in pt3D:
            mylist.append(round(pt,0))
        if(tuple(mylist) not in myset):
            myset.add(tuple(mylist))
            
def unpack_sklearn_query_radii(Stars, neighbors, counts):
    tooClose = []
    for pair in neighbors[counts > 1]:
        tooClose.append(Stars[pair[0]])
        tooClose.append(Stars[pair[1]])
    return np.array(tooClose)

XenoSupermanGalaxy = read_dictionary('XenoSupermanGalaxy.pkl')
GFFA = read_dictionary('GFFA.pkl')

plt.style.use('dark_background')
#plt.scatter(Arms[:,0], Arms[:,1], c='thistle', s = .1, alpha = 1)
#plt.scatter(CenterGlob[:,0], CenterGlob[:,1], c = 'indigo', s = .1, alpha = .7)
# plt.grid()
# plt.scatter(GFFA['Stars'][:,0], GFFA['Stars'][:,1], c = 'blue', s = .1, alpha = .2)
# plt.style.use('dark_background')
    
# dataset will be subset of stars from each galaxy.
# But first lets combine both galaxies into one large colliosn dataset
TrainingSize = min(len(GFFA['Stars']), len(XenoSupermanGalaxy['Stars'] ) ) 



collision = dict()
collision['Arms'] = np.vstack((GFFA['Arms'].copy(), XenoSupermanGalaxy['Arms'].copy()))
collision['CenterGlob'] = np.vstack((GFFA['CenterGlob'].copy(), XenoSupermanGalaxy['CenterGlob'].copy()))
collision['Stars'] = np.vstack((GFFA['Stars'].copy(), XenoSupermanGalaxy['Stars'].copy()))
collision['Stars'].shape

# get the index of the stars to use from XenoSupermanGalaxy
XenoIndex = np.random.choice(len(XenoSupermanGalaxy['Stars']), TrainingSize, replace=False)
# get the index of the stars to use from GFFAIndex
GFFAIndex = np.random.choice(len(GFFA['Stars']), TrainingSize, replace=False)

# create a list with a label for each item in the combined training set
# the first hald of the list indicates that class 0 will be for GFFA, 1 will be XenoSupermanGalaxy
y = [0]*TrainingSize + [1]*TrainingSize
# Stack the stars subset in same order as the labels, GFFA first, XenoSupermanGalaxy second
trainGalaxy = np.vstack((GFFA['Stars'][GFFAIndex], XenoSupermanGalaxy['Stars'][XenoIndex]))  


In [ ]:
trainGalaxy.shape

In [ ]:
%%time
Radius = 3

tree = KDTree(GFFA['Stars'], leaf_size = 42)              
neighbors = tree.query_radius(GFFA['Stars'],  Radius) 
counts = tree.query_radius(GFFA['Stars'],  Radius, count_only = 1)
GFFA['tooClose']  = unpack_sklearn_query_radii(GFFA['Stars'], neighbors, counts)

tree = KDTree(XenoSupermanGalaxy['Stars'], leaf_size = 42)              
neighbors = tree.query_radius(XenoSupermanGalaxy['Stars'],  Radius) 
counts = tree.query_radius(XenoSupermanGalaxy['Stars'],  Radius, count_only = 1)
XenoSupermanGalaxy['tooClose']  = unpack_sklearn_query_radii(XenoSupermanGalaxy['Stars'], neighbors, counts)


collision = dict()  
collision['Arms'] = np.vstack((GFFA['Arms'].copy(), XenoSupermanGalaxy['Arms'].copy()))
collision['CenterGlob'] = np.vstack((GFFA['CenterGlob'].copy(), XenoSupermanGalaxy['CenterGlob'].copy()))
collision['Stars'] = np.vstack((GFFA['Stars'].copy(), XenoSupermanGalaxy['Stars'].copy())).copy()
tree = KDTree(collision['Stars'], leaf_size = 42)  

#kdtree  tree.query_radius

neighbors = tree.query_radius(collision['Stars'],  Radius) 
counts = tree.query_radius(collision['Stars'],  Radius, count_only = 1)
collision['tooClose']  = unpack_sklearn_query_radii(collision['Stars'], neighbors, counts)

print("GFFA['Stars'].shape ", GFFA['Stars'].shape)
print("XenoSupermanGalaxy['Stars'].shape ", XenoSupermanGalaxy['Stars'].shape)
print("collision['Stars'].shape ", collision['Stars'].shape)

print("\nGFFA['tooClose'].shape ", GFFA['tooClose'].shape)
print("XenoSupermanGalaxy['tooClose'].shape ", XenoSupermanGalaxy['tooClose'].shape)
print("collision['tooClose'].shape ", collision['tooClose'].shape)

# Find Glactic collision near misses

In [ ]:
          
GFFA['tooClose']
GFFAset = set()           
add(GFFA['tooClose'], GFFAset)
print('GFFAset ', len(GFFAset))
XenoSupermanGalaxy['tooClose']
XenoSupermanGalaxyset = set()           
add(XenoSupermanGalaxy['tooClose'], XenoSupermanGalaxyset)
print('XenoSupermanGalaxyset ', len(XenoSupermanGalaxyset))

collision['tooClose']
collisionset = set()           
add(collision['tooClose'], collisionset)
print('collisionset ', len(collisionset))

GFFAStarsSet = set()           
add(GFFA['Stars'], GFFAStarsSet)

XenoSupermanGalaxyStarsSet = set()           
add(XenoSupermanGalaxy['Stars'], XenoSupermanGalaxyStarsSet)

GalacticCollision = collisionset - XenoSupermanGalaxyset - GFFAset
print( 'Galactic Collisions list of star locations where distance less than {} light years : {}'.format(Radius, GalacticCollision))

# Use python sets to find interacting points

Essentially, for each galaxy individually, we find all existing stars within three lights years within a single glaxay. Next find all stars within three light years in the combined galaxy. To find all stars in one galaxy that are within three light years of a star in a different galaxy perform the set difference as follows:

**GalacticCollision = collisionset - XenoSupermanGalaxyset - GFFAset**

Essentially remove the known tooClose from the collionset and what remains are the singletons from each galaxy that are tooClose after combining

# Plot Star Positions in Red

For those stars from one galaxy that are within three light years of a star in another galaxy

In [ ]:
# # Plot galaxy matplotlib
%matplotlib inline
# Import libraries
from mpl_toolkits import mplot3d
import numpy as np
import matplotlib.pyplot as plt
 
# Creating figure
fig = plt.figure(figsize = (10, 10))
ax = plt.axes(projection ="3d")

ax.set_xticks([-20000, -10000, 0, 10000, 20000])
ax.set_yticks([-20000, -10000, 0, 10000, 20000])
ax.set_zticks([-20000, -10000, 0, 10000, 20000])

# ax.axes.set_xlim3d(left=0.2, right=9.8) 
# ax.axes.set_ylim3d(bottom=0.2, top=9.8) 
#ax.axes.set_zlim3d(bottom=-5000, top=5000) 

# Creating plot
GalacticCollisionsNP = np.array(list(GalacticCollision))

ax.scatter3D(GFFA['Stars'][::5,0], 
             GFFA['Stars'][::5,1], 
             GFFA['Stars'][::5,2], color = "blue", s = .2, alpha = 0)

ax.scatter3D(XenoSupermanGalaxy['Stars'][::5,0], 
             XenoSupermanGalaxy['Stars'][::5,1], 
             XenoSupermanGalaxy['Stars'][::5,2], color = "g", s = .2, alpha = 0 )

ax.scatter3D(GalacticCollisionsNP[:,0], 
             GalacticCollisionsNP[:,1], 
             GalacticCollisionsNP[:,2], color = "r", s = 50, alpha = 1)

plt.title("Stars within {} light years".format(Radius))
plt.style.use('dark_background')
ax.w_xaxis.pane.fill = False
ax.w_yaxis.pane.fill = False
ax.w_zaxis.pane.fill = False

# show plot
plt.show()

# Notices & Disclaimers

Intel technologies may require enabled hardware, software or service activation.

No product or component can be absolutely secure.
Your costs and results may vary.

© Intel Corporation. Intel, the Intel logo, and other Intel marks are trademarks of Intel Corporation or its subsidiaries. 
*Other names and brands may be claimed as the property of others.
